In [10]:
# Get image data w/ nibabel
import nibabel as nib
import numpy as np
def get_data(filename):
    img = nib.load(filename)
    return img.get_data()
def scale(array):
    return (array-np.min(array))/(np.max(array) - np.min(array))    
def load(s):
    return get_data('../data/case_%s_2.nii.gz'%s).transpose((2,0,1)), get_data('../data/case_%s_labels.nii.gz'%s).transpose((2,0,1))
mri_data = scale(get_data('../data/case_002_2.nii.gz')).transpose((2,0,1))
labelled_data = get_data('../data/case_002_labels.nii.gz').transpose((2,0,1))

In [12]:
a,b = load('001')

In [22]:
eh = np.arange(10,1,-1)
eh[[3,1,5]]

array([7, 9, 5])

In [97]:
import nibabel as nib
import numpy as np
from scipy.misc import imresize
def get_data(filename):
    img = nib.load(filename)
    return img.get_data()
def scale(array):
    return (array-np.min(array))/(np.max(array) - np.min(array))    
def load(s):
    return get_data('../data/case_%s_2.nii.gz'%s).transpose((2,0,1)), get_data('../data/case_%s_labels.nii.gz'%s).transpose((2,0,1))
def preprocess(mri_slice):
    img0 = imresize(mri_slice, (224,224))
    img0.resize((img0.shape[0],img0.shape[1],1))
    img0 = img0.transpose((2,0,1))
    return np.expand_dims(img0, axis=0)
def get_testing_data(s):
    x, y = load(s)
    
    x = scale(x)
    
    x1 = np.zeros((x.shape[0],1,224, 224,))
    y1 = np.zeros(x1.shape)
    
    for n in range(x.shape[0]):
        x1[n] = preprocess(x[n])
        y1[n] = preprocess(y[n])
    return x1, y1
def get_training_data(s, slice_filter="equal"):
    print("Getting data for",s)
    x, y = load(s)
    
    x_sums_by_slice = x.sum(axis=(1,2)) # used to avoid completely blank slices
    y_sums_by_slice = y.sum(axis=(1,2))
    
    # Get slice #s for slices with/out patches
    patch_slices = np.where(y_sums_by_slice>0)[0]
    non_patch_slices = np.where((y_sums_by_slice==0) & (x_sums_by_slice>0))[0]
    np.random.shuffle(non_patch_slices)
        
    # Filter
    if slice_filter == "equal":
        #print (non_patch_slices)
        non_patch_slices = non_patch_slices[:patch_slices.size]
        #print (non_patch_slices)
    if slice_filter == "patches_only":
        non_patch_slices = []
    
    all_slices = np.concatenate((patch_slices,non_patch_slices)).astype(int)
    np.random.shuffle(all_slices)
    
    
    # preprocess data    
    x = scale(x)
    
    x1 = np.zeros((all_slices.size,1,224, 224,))
    y1 = np.zeros(x1.shape)
    
    for n,slice_n in enumerate(all_slices):
        x1[n] = preprocess(x[slice_n])
        y1[n] = preprocess(y[slice_n])
    
    return x1, y1

In [77]:
c,d=get_training_data('001')
e,f=get_training_data('002')

[17  3  4  5  1  9 20 16 11 13  7  6 12  2  8 31 32 10 15 19 14  0 21 18]
[17  3  4  5  1  9 20 16 11]
[10 19  9 15 11  2 14 26 16 12  1  4 23  7 25  8  3  5 24 18  0 17 21 13  6]
[10 19]


In [80]:
g = np.vstack((c,e))

(22, 1, 224, 224)

In [66]:
d.shape

(18, 1, 224, 224)

In [82]:
def get_case_num(name):    
    return name[name.find("case_")+len("case_"):name.find("_labels")]

import glob
i = 0

all_cases = []
for name in glob.glob('../data/*labels.nii.gz'):
    i+=1
    print(name,get_case_num(name))
    all_cases.append(get_case_num(name))
    
np.random.shuffle(all_cases)

# pick 20 training. The rest are 
train_cases = all_cases[:20]
test_cases = all_cases[20:]

#train_cases.sort()
#test_cases.sort()
print ("Training cases: ",', '.join(sorted(train_cases)))
print ("Testing cases: ",', '.join(sorted(test_cases)))

../data/case_021_labels.nii.gz 021
../data/case_011_labels.nii.gz 011
../data/case_018_labels.nii.gz 018
../data/case_047_labels.nii.gz 047
../data/case_055_labels.nii.gz 055
../data/case_035_labels.nii.gz 035
../data/case_007_labels.nii.gz 007
../data/case_039_labels.nii.gz 039
../data/case_019_labels.nii.gz 019
../data/case_008_labels.nii.gz 008
../data/case_053_labels.nii.gz 053
../data/case_004_labels.nii.gz 004
../data/case_058_labels.nii.gz 058
../data/case_028_labels.nii.gz 028
../data/case_017_labels.nii.gz 017
../data/case_010_labels.nii.gz 010
../data/case_002_labels.nii.gz 002
../data/case_034_labels.nii.gz 034
../data/case_014_labels.nii.gz 014
../data/case_001_labels.nii.gz 001
../data/case_046_labels.nii.gz 046
../data/case_009_labels.nii.gz 009
../data/case_005_labels.nii.gz 005
../data/case_013_labels.nii.gz 013
../data/case_052_labels.nii.gz 052
../data/case_003_labels.nii.gz 003
Training cases:  001, 002, 003, 004, 005, 007, 010, 011, 013, 017, 018, 019, 028, 034, 035

In [128]:
train_data,train_labels = get_training_data(train_cases[0],'all')
for i in train_cases[1:]:
    data, labels = get_training_data(i,'all')
    train_data = np.vstack((train_data,data))
    train_labels = np.vstack((train_labels,labels))

Getting data for 053
Getting data for 052
Getting data for 019
Getting data for 034
Getting data for 035
Getting data for 017
Getting data for 002
Getting data for 058
Getting data for 010
Getting data for 055
Getting data for 039
Getting data for 007
Getting data for 013
Getting data for 028
Getting data for 003
Getting data for 001
Getting data for 005
Getting data for 011
Getting data for 018
Getting data for 004


In [129]:
test_data, test_labels = get_training_data(test_cases[0],'all')
for i in test_cases[1:]:
    data, labels = get_training_data(i,'all')
    test_data = np.vstack((test_data,data))
    test_labels = np.vstack((test_labels,labels))

Getting data for 047
Getting data for 021
Getting data for 014
Getting data for 008
Getting data for 046
Getting data for 009


In [126]:
test_data.shape

(67, 1, 224, 224)

In [113]:
t.shape

(6,)

In [130]:
import h5py

# Shuffle training data
p = np.random.permutation(len(train_data))
p2 = np.random.permutation(len(test_data))
with h5py.File('dataset_1_all.h5', 'w') as hf:
    hf.create_dataset("train_x",  data=train_data[p])
    hf.create_dataset("train_y",  data=train_labels[p])
    hf.create_dataset("test_x",  data=test_data[p2])
    hf.create_dataset("test_y",  data=test_labels[p2])